In [2]:
import os
import pyodbc
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
def get_db_connection(DB_NAME:str):
    conn = pyodbc.connect(
        f"DRIVER={os.getenv('DB_DRIVER')};"
        f"SERVER={os.getenv('DB_SERVER')};"
        f"DATABASE={DB_NAME};"
        f"UID={os.getenv('DB_UID')};"
        f"PWD={os.getenv('DB_PWD')};"
        f"TrustServerCertificate={os.getenv('TRUST_SERVER_CERTIFICATE')};"
    )
    
    return conn


In [5]:
role_df = pd.read_csv('./data/role.csv', encoding='utf-8')
permission_df = pd.read_csv('./data/permission.csv', encoding='utf-8')
role_permission_df = pd.read_csv('./data/role_permission.csv', encoding='utf-8')

In [6]:
DB_NAME = os.getenv('DB_NAME')
conn = get_db_connection(DB_NAME)

cursor = conn.cursor()

Create role, permission, role_permission

In [7]:
cursor.execute("""
    IF OBJECT_ID('Role_Permission', 'U') IS NOT NULL DROP TABLE Role_Permission;
    IF OBJECT_ID('Permission', 'U') IS NOT NULL DROP TABLE Permission;
    IF OBJECT_ID('Role', 'U') IS NOT NULL DROP TABLE Role;
""")

In [8]:
cursor.execute("""
    CREATE TABLE [role] (
        id INT PRIMARY KEY IDENTITY(1,1),
        name NVARCHAR(255) NOT NULL,
        status NVARCHAR(50) DEFAULT 'Active'
    )
""")

In [9]:
cursor.execute("""
    CREATE TABLE permission (
        id INT PRIMARY KEY IDENTITY(1,1),
        name NVARCHAR(255) NOT NULL,
    )
""")

In [10]:
cursor.execute("""
    CREATE TABLE role_permission (
        role_id INT,
        permission_id INT,
        CONSTRAINT pk_role_permission PRIMARY KEY (role_id, permission_id),
        CONSTRAINT fk_role_permission_role FOREIGN KEY (role_id) REFERENCES [role](id),
        CONSTRAINT fk_role_permission_permission FOREIGN KEY (permission_id) REFERENCES permission(id)
    );
""")

In [ ]:
role_tuples = [
    (row['id'], row['name'], row['status']) for _, row in role_df.iterrows()
]

permission_tuples = [
    (row['id'], row['name']) for _, row in permission_df.iterrows()
]

role_permission_tuples = [
    (int(row['role_id']), int(row['permission_id'])) for _, row in role_permission_df.iterrows()
]

In [12]:
cursor.execute("SET IDENTITY_INSERT [role] ON;")
cursor.executemany("INSERT INTO [role] (id, name, status) VALUES (?, ?, ?)", role_tuples)
cursor.execute("SET IDENTITY_INSERT [role] OFF;")

cursor.execute("SET IDENTITY_INSERT [permission] ON;")
cursor.executemany("INSERT INTO permission (id, name) VALUES (?, ?)", permission_tuples)
cursor.execute("SET IDENTITY_INSERT [permission] OFF;")

cursor.executemany("INSERT INTO role_permission (role_id, permission_id) VALUES (?, ?)", role_permission_tuples)

In [ ]:
conn.commit()
conn.close()